### ** Bio606 Python Training Course **
##### _ author: Y.F.Liu, Ph.D. _
##### _ date: 2017.11.1 _

#### Step 1 - read input DNA sequence from plain text file 'TP53.txt'

In [ ]:
import re, csv,  pprint

def read_seq(fn):
    """讀取 DNA 序列"""
    with open(fn, 'rt') as f:
        seq = list()      
        for line in f.readlines():
            if not re.match('^>', line):   # remove description lines
                for base in line:
                    if not base == '\n':
                        seq.append(base)              
    
    return seq

seq = read_seq('TP53.txt')
seq?

In [ ]:
%%perl
open ($fn, "TP53.txt");
while ($line=<$fn>) {
  chomp($line);
  if ($line !~ /^>/) {
     foreach $base (split("", $line)) {
        push(@seq, $base);
     }
  }
}
print "@seq\n";
close ($fn);

#### Step 2 - DNA complementary sequence from seq

In [ ]:
def complementary_seq(seq):
    """互補 DNA 序列"""
    c_seq = list()

    c_table = {'A':'T', 'T':'A', 'C':'G', 'G':'C'}
    
    for base in seq:
        c_seq.append(c_table[base])

    return c_seq

c_seq = complementary_seq(seq)
c_seq?

In [ ]:
%%perl
open ($fn, "TP53.txt");
while ($line=<$fn>) {
  chomp($line);
  if ($line !~ /^>/) {
     foreach $base (split("", $line)) {
        push(@seq, $base);
     }
  }
}
close ($fn);

%c_table = ('A'=>'T', 'T'=>'A', 'C'=>'G', 'G'=>'C');

foreach $base (@seq) {
   push(@c_seq, $c_table{$base});
}
print "@c_seq\n";

#### Step 3 - Reverse the DNA sequence from c_seq    

In [ ]:
def reverse_seq(c_seq): 
    """反轉 DNA 序列"""
    c_seq.reverse()
    
    return c_seq

r_seq = reverse_seq(c_seq)
r_seq?

In [ ]:
%%perl
open ($fn, "TP53.txt");
while ($line=<$fn>) {
  chomp($line);
  if ($line !~ /^>/) {
     foreach $base (split("", $line)) {
        push(@seq, $base);
     }
  }
}
close ($fn);

%c_table = ('A'=>'T', 'T'=>'A', 'C'=>'G', 'G'=>'C');

foreach $base (@seq) {
   push(@c_seq, $c_table{$base});
}

@r_seq = reverse(@c_seq);
print "@r_seq\n"

#### Step 4 - Create the codon table of DNA -> Protein from csv file 'T_table.csv'

In [ ]:
def read_t_table(fn):
    """建立 Translation Table"""
    with open(fn, 'r') as f:
        t_table = dict()
        for words in csv.reader(f, delimiter = '\t'):
            t_table[words[0]] = {'one':words[1], 'three':words[2]}
                
    return t_table

t_table = read_t_table('T_table.csv')
t_table?

In [ ]:
%%perl
open ($fn, "T_table.csv");
$ref = {};
while ($line=<$fn>) {
  chomp($line);
  my @words = split("\t", $line);
     $ref->{$words[0]}->{'one'} = $words[1];
     $ref->{$words[0]}->{'three'} = $words[2];
}
foreach $codon (keys %{$ref}) {
  print $codon."\t".$ref->{$codon}->{'one'}."\n"
}
close ($fn);

#### Step 5 - Translation the six reading frames from DNA double strains 

In [ ]:
def translation(seq, t_table):
    """進行轉譯 DNA 序列 (包含正反兩股的 6 個 Frames)"""
    codon = str()
    all = dict()
    
    for frame in range(3):
      protein = list()
      for item in range(frame, int(len(seq)/3)*3, 3):
        protein.append(t_table[codon.join(seq[item:item+3])]['one'])
      all['positive strain frame-'+str(frame+1)] = protein
    
    c_seq = complementary_seq(seq)
    r_seq = reverse_seq(c_seq)

    for frame in range(3):
      protein = list()
      for item in range(frame, int(len(r_seq)/3)*3, 3):
        protein.append(t_table[codon.join(seq[item:item+3])]['one'])
      all['negative strain frame-'+str(frame+1)] = protein
    return all

frames = translation(seq, t_table)
frames?

In [ ]:
%%perl
open ($fn, "TP53.txt");
while ($line=<$fn>) {
  chomp($line);
  if ($line !~ /^>/) {
     foreach $base (split("", $line)) {
        push(@seq, $base);
     }
  }
}

%c_table = ('A'=>'T', 'T'=>'A', 'C'=>'G', 'G'=>'C');
foreach $base (@seq) {
   push(@c_seq, $c_table{$base});
}

@r_seq = reverse(@c_seq);

open ($fn, "T_table.csv");
$T_table = {};
while ($line=<$fn>) {
  chomp($line);
  my @words = split("\t", $line);
     $T_table->{$words[0]}->{'one'} = $words[1];
     $T_table->{$words[0]}->{'three'} = $words[2];
}
close ($fn);

$frames = {};
foreach my $frame (1..3) {
   for ($i=$frame; $i<($#seq); $i +=3) {
      my $codon = $seq[$i].$seq[$i+1].$seq[$i+2];
      $frames->{'positive strain frame-'.$frame} .= $T_table->{$codon}->{'one'}
   }
}
foreach my $frame (1..3) {
   for ($i=$frame; $i<($#r_seq); $i +=3) {
      my $codon = $r_seq[$i].$r_seq[$i+1].$r_seq[$i+2];
      $frames->{'negative strain frame-'.$frame} .= $T_table->{$codon}->{'one'}
   }
}

foreach my $item (keys %{$frames}) {
   print $item."\t".$frames->{$item}."\n";
}

#### Step 6 - Determine the ORFs and generate info of these ORFs (longest orf and its lenght)

In [ ]:
def determine_orf(frames):
    """ 判斷每個 Frames 的開放式框架 (ORFs)，
        記錄每個 Frames 上所有的開放式框架，和最長的開放式框架與長度 """
    all_orfs = dict()
    
    for info, protein in frames.items():
      orfs = list()
      longest = 0
      num = 0
      candidate = 0
      for pp in ''.join(protein).split('*'):
        for x in range(len(pp)):
          if (pp[x] == 'M'):
            orfs.append(pp[x:])
            if (len(pp)-x+1) > longest:
              longest = len(pp)-x+1
              candidate = num
            num += 1  
            break
      all_orfs[info] = {'orfs': orfs,'candidate': candidate, 'longest': longest}
      
    return all_orfs

all_orfs = determine_orf(frames)
all_orfs?

In [ ]:
%%perl
open ($fn, "TP53.txt");
while ($line=<$fn>) {
  chomp($line);
  if ($line !~ /^>/) {
     foreach $base (split("", $line)) {
        push(@seq, $base);
     }
  }
}

%c_table = ('A'=>'T', 'T'=>'A', 'C'=>'G', 'G'=>'C');
foreach $base (@seq) {
   push(@c_seq, $c_table{$base});
}

@r_seq = reverse(@c_seq);

open ($fn, "T_table.csv");
$T_table = {};
while ($line=<$fn>) {
  chomp($line);
  my @words = split("\t", $line);
     $T_table->{$words[0]}->{'one'} = $words[1];
     $T_table->{$words[0]}->{'three'} = $words[2];
}
close ($fn);

$frames = {};
foreach my $frame (1 .. 3) {
   for ($i=$frame; $i<($#seq); $i += 3) {
      my $codon = $seq[$i].$seq[$i+1].$seq[$i+2];
      $frames->{'positive strain frame-'.$frame} .= $T_table->{$codon}->{'one'};
   }
}
foreach my $frame (1 .. 3) {
   for ($i=$frame; $i<($#r_seq); $i += 3) {
      my $codon = $r_seq[$i].$r_seq[$i+1].$r_seq[$i+2];
      $frames->{'negative strain frame-'.$frame} .= $T_table->{$codon}->{'one'};
   }
}

$all_orfs = {};

foreach my $info (keys %{$frames}) {
#  print $frames->{$info},"\n";
#  print length($frames->{$info})."\n";
  $longest = 0;
  $candidate = 0;
  $num = 0;
  foreach $pp (split(/\*/, $frames->{$info})) {
    @seq = split("", $pp);
    $flag = $num;
    foreach $x (0 .. $#seq) {
      if (($seq[$x] eq "M") and ($flag == $num)){
        if ((length($pp)-$x+1) > $longest) {
          $longest = length($pp)-$x+1;
          $candidate = $num;
        }
        push(@{$all_orfs->{$info}->{'orf'}}, $pp);
#        print $pp."\n";
        $num += 1;
      }
    }
  }
  $all_orfs->{$info}->{'candidate'} = $candidate;
  $all_orfs->{$info}->{'longest'} = $longest;  
}

foreach my $info (keys %{$all_orfs}) {
   print $info."\n";
   print $all_orfs->{$info}->{'candidate'}."\n";
   print $all_orfs->{$info}->{'longest'}."\n";
   foreach $pp (@{$all_orfs->{$info}->{'orf'}}) {
      print $pp."\n";
   }
   print "\n";
}

#### Step 7 - Select the ORF for the gene (TP53)         

In [ ]:
def select_orf(all):
    """選擇正確的 Protein 框架"""
    pp = pprint.PrettyPrinter(depth=3)
    pp.pprint(all)

select_orf(all_orfs)

In [ ]:
%%perl 
foreach my $x (1..9) {
    foreach my $y (1..9) {
       print $x."x".$y."= ".$x*$y."\t";
    }
    print "\n";
}

In [ ]:
%%perl

@list = (1, 2, 3, 4, 5, 6);

print $list[2];